# Using MLRUN with Nuclio

In [1]:
# nuclio: ignore
import nuclio

In [20]:
from mlrun import get_or_create_ctx, run_start
import time

def handler(context, event):
    ctx = get_or_create_ctx('myfunc', event=event)
    p1 = ctx.get_param('p1', 1)
    p2 = ctx.get_param('p2', 'a-string')

    context.logger.info(
        f'Run: {ctx.name} uid={ctx.uid}:{ctx.iteration} Params: p1={p1}, p2={p2}')

    time.sleep(1)

    # log scalar values (KFP metrics)
    ctx.log_result('accuracy', p1 * 2)
    ctx.log_result('latency', p1 * 3)

    # log various types of artifacts (and set UI viewers)
    ctx.log_artifact('test.txt', body=b'abc is 123')
    ctx.log_artifact('test.html', body=b'<b> Some HTML <b>', viewer='web-app')

    context.logger.info('run complete!')
    return ctx.to_json()

In [ ]:
# nuclio: ignore
resp = run_start({}, handler=handler)

In [37]:
# nuclio: ignore
spec = nuclio.ConfigSpec(env={'MYENV_VAR': 'something'}, 
                         cmd=['pip install git+https://github.com/v3io/mlrun.git'],
                         config={'spec.build.baseImage': 'python:3.6-jessie',
                                 'spec.triggers.web': {'kind': 'http', 'maxWorkers': 8}},
                         mount=nuclio.Volume('User','~/'))

addr = nuclio.deploy_file('',name='myfunc', project='mlrun', verbose=False, spec=spec)

[nuclio.deploy] 2019-07-29 23:06:22,077 (info) Building processor image
[nuclio.deploy] 2019-07-29 23:06:28,119 (info) Pushing image
[nuclio.deploy] 2019-07-29 23:06:28,120 (info) Build complete
[nuclio.deploy] 2019-07-29 23:06:31,143 done updating myfunc, function address: 18.195.168.255:31426


In [36]:
# nuclio: ignore
resp = run_start({}, command=f'http://{addr}')

2019-07-29 22:59:26.534653  info   Run: myfunc uid=33fc00aa2303429987759c73fb325098:0 Params: p1=1, p2=a-string  
2019-07-29 22:59:27.535268  info   run complete!  



uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
...325098,0,Jul 29 22:59:26,completed,myfunc,owner=iguaziohost=myfunc-6f49865498-656r4runtime=remote,,p1=1p2=a-string,accuracy=2latency=3,test.txttest.html
